In [32]:
import os
os.environ['CUDA_HOME'] = "/usr/local/cuda-12.4"
print(os.environ['CUDA_HOME'])

/usr/local/cuda-12.4


In [33]:
print(os.environ['PATH'])

/home/mjknowles/anaconda3/envs/rag-from-scratch/bin:/home/mjknowles/.vscode-server/bin/863d2581ecda6849923a2118d93a088b0745d9d6/bin/remote-cli:/home/mjknowles/.local/bin:/usr/local/cuda-12.4/bin:/home/mjknowles/anaconda3/envs/rag-from-scratch/bin:/home/mjknowles/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/WINDOWS/system32:/mnt/c/WINDOWS:/mnt/c/WINDOWS/System32/Wbem:/mnt/c/WINDOWS/System32/WindowsPowerShell/v1.0:/mnt/c/WINDOWS/System32/OpenSSH:/mnt/c/Program Files/Docker/Docker/resources/bin:/mnt/c/Program Files/Git/cmd:/mnt/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/mnt/c/Users/mjkno/AppData/Local/Microsoft/WindowsApps:/mnt/c/Users/mjkno/AppData/Local/Programs/Microsoft VS Code/bin:/snap/bin


In [34]:
# %pip install -r requirements.txt

In [35]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
#%pip install llama-index-llms-llama-cpp

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 CUDAFLAGS="-arch=all -lcublas" pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir --verbose

In [63]:
#cpu
#%env CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"
#cuda
#%env CMAKE_ARGS="-DLLAMA_CUBLAS=on" 
#%env FORCE_CMAKE=1
#%env CUDAFLAGS="-arch=all -lcublas"
#%pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir --verbose

env: CMAKE_ARGS="-DLLAMA_CUBLAS=on"
env: FORCE_CMAKE=1
env: "CUDAFLAGS=-arch=all -lcublas"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Using pip 23.3.1 from /home/mjknowles/anaconda3/envs/rag-from-scratch/lib/python3.11/site-packages/pip (python 3.11)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 13.9 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ...   Running command pip subprocess to install build dependencies
    Using cached scikit_build_core-0.8.2-py3-none-any.whl.metadata (19 kB)
    Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
    Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
    Using cached pyproject_metadata-0.7.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached scikit_build_core-0.8.2-py3-none-any.whl (140 kB)
  Using cached packaging-24.0-py3-none-any.whl (53 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl (31 kB)
  Using cached pyproject_metadata-0.7.1-py3-none-any.whl (7.4 kB)
done
  Getting requirements to build wheel ...   Running command Getting requirements to build wheel
done
  Installing backend dependencies ...   Running command pip

In [ ]:
%pip install llama-index-llms-llama-cpp

In [37]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [38]:
#from huggingface_hub import hf_hub_download
#hf_hub_download(repo_id='TheBloke/Mistral-7B-v0.1-GGUF', filename='mistral-7b-v0.1.Q4_K_M.gguf', local_dir='./models', local_dir_use_symlinks=False)

In [39]:
from llama_index.llms.llama_cpp import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
model_url = "https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/resolve/main/mistral-7b-v0.1.Q4_K_M.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=None,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path='models/mistral-7b-v0.1.Q4_K_M.gguf',
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 15000},
    verbose=True,
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/mistral-7b-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_

In [40]:
%pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [41]:
# import psycopg2

# db_name = "vector_db"
# host = "localhost"
# password = "postgres"
# port = "5432"
# user = "postgres"
# # conn = psycopg2.connect(connection_string)
# conn = psycopg2.connect(
#     dbname=postgres,
#     host=host,
#     password=password,
#     port=port,
#     user=user,
# )
# conn.autocommit = True

# with conn.cursor() as c:
#     c.execute(f"DROP DATABASE IF EXISTS {db_name}")
#     c.execute(f"CREATE DATABASE {db_name}")


In [42]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

db_name = "vectordb"
# host = "db"
host = "localhost"
password = "testpwd"
port = "5432"
user = "testuser"
vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

In [43]:
#%mkdir data
#%wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

In [44]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

In [45]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))



In [46]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [47]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding


In [48]:
vector_store.add(nodes)

['cd48d839-0870-4881-9e99-089cc527daaf',
 'f9a104a3-c04c-4eb8-a0b0-cef35d99076f',
 '455f7c23-b980-456f-b952-b30f19d7494a',
 'c26d4d6c-56d1-484f-8726-009e930d749d',
 '41dae9b9-b904-4063-8fce-0045eed1da79',
 '17e612b3-a576-4022-a6b4-d3463d889111',
 'bbe3ed3f-d062-46cd-b3fd-071f30dc47a7',
 'cef9c441-3e02-49d6-81dc-9dd593738a70',
 '5984fb4a-5f14-4f65-8f11-cd9a97603b60',
 '4d591297-e628-4756-94e3-6f545fa7a098',
 '649db18a-8e43-40d0-b25e-7397fc53baa7',
 'ff79b38e-4bf6-4735-afa0-696246a83d19',
 '3626ea86-86cb-4689-97e9-ef8c599a2f77',
 '1b4a635c-8e2f-427f-aa14-312df8732543',
 'cbf263f4-1c2b-4281-86a4-edd20a2a51c2',
 '442e080e-ccbb-4eca-8ccf-131e9d4c76fb',
 '108f9853-b1a5-46d9-9b38-177a80082701',
 '0b6d06e8-a812-4e3c-863a-124790a43bc7',
 '64437939-ae03-4e84-9d8f-8d5445cf2cf8',
 'df09c129-323f-49be-b1b7-49ad5d10de1c',
 '94bfaa87-3d92-4fa0-b818-1ad991b86965',
 '18dda521-5faa-4f16-9e91-f6f687543cba',
 '0aa2bb2a-39fc-48e5-8872-b988f2d7ba18',
 'ea685646-81c1-4b25-88a5-3100a4977152',
 'e10ebe22-75a8-

In [49]:
query_str = "Can you tell me about the key concepts for safety finetuning"

In [50]:
query_embedding = embed_model.get_query_embedding(query_str)

In [51]:
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [52]:
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())


TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary

In [53]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))


In [54]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [55]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [56]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm, streaming=True)

In [57]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)
response.print_response_stream()

1. Llama 2 performs better than other open-source models on most benchmarks. 2. Llama 2 performs
close to GPT-3.5 on MMLU and GSM8K, but there is a significant gap on coding benchmarks. 3. Llama 2
results are on par or better than PaLM on almost all benchmarks. 4. There is still a large gap in performance
between Llama 2 and GPT-4 and PaLM-2-L.


llama_print_timings:        load time =   23013.69 ms
llama_print_timings:      sample time =      18.31 ms /   115 runs   (    0.16 ms per token,  6282.09 tokens per second)
llama_print_timings: prompt eval time =   35743.21 ms /   764 tokens (   46.78 ms per token,    21.37 tokens per second)
llama_print_timings:        eval time =   29814.13 ms /   114 runs   (  261.53 ms per token,     3.82 tokens per second)
llama_print_timings:       total time =   65801.67 ms /   878 tokens


In [58]:
print(str(response))


1. Llama 2 performs better than other open-source models on most benchmarks. 2. Llama 2 performs
close to GPT-3.5 on MMLU and GSM8K, but there is a significant gap on coding benchmarks. 3. Llama 2
results are on par or better than PaLM on almost all benchmarks. 4. There is still a large gap in performance
between Llama 2 and GPT-4 and PaLM-2-L.


In [59]:
print(response.source_nodes[0].get_content())


Additionally, Llama 2 70B model outperforms all open-source models.
In addition to open-source models, we also compare Llama 2 70B results to closed-source models. As shown
in Table 4, Llama 2 70B is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant
gap on coding benchmarks. Llama 2 70B results are on par or better than PaLM (540B) (Chowdhery et al.,
2022) on almost all benchmarks. There is still a large gap in performance between Llama 2 70B and GPT-4
and PaLM-2-L.
We also analysed the potential data contamination and share the details in Section A.6.
Benchmark (shots)
GPT-3.5
GPT-4
PaLM
PaLM-2-L
Llama 2
MMLU (5-shot)
70.0
86.4
69.3
78.3
68.9
TriviaQA (1-shot)
–
–
81.4
86.1
85.0
Natural Questions (1-shot)
–
–
29.3
37.5
33.0
GSM8K (8-shot)
57.1
92.0
56.5
80.7
56.8
HumanEval (0-shot)
48.1
67.0
26.2
–
29.9
BIG-Bench Hard (3-shot)
–
–
52.3
65.7
51.2
Table 4: Comparison to closed-source models on academic benchmarks. Results for GPT-3.5 and GPT-4
are from OpenAI

In [60]:
query_str = "What is the parameter count of a Llama 2 model?"
response = query_engine.query(query_str)
response.print_response_stream()

Llama.generate: prefix-match hit


7B, 13B, and 70B


llama_print_timings:        load time =   23013.69 ms
llama_print_timings:      sample time =       2.40 ms /    14 runs   (    0.17 ms per token,  5835.76 tokens per second)
llama_print_timings: prompt eval time =   50359.80 ms /   997 tokens (   50.51 ms per token,    19.80 tokens per second)
llama_print_timings:        eval time =    3810.49 ms /    13 runs   (  293.11 ms per token,     3.41 tokens per second)
llama_print_timings:       total time =   54207.05 ms /  1010 tokens


In [61]:
print(str(response))


7B, 13B, and 70B
